In [ ]:
# @title Default title text
pip install DeepFace


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=9e80f21a4ec89dce479676084a4f02d361edc5bb351284ef54a601e132b9069f
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [78]:
from itertools import islice
import time


In [ ]:

from deepface import DeepFace


def emotion_faces():
    """
    Determines the emotions of a collection of faces in a frame

    :param list faces: a numpy array in BGR format, base64 encoded image, or path to the image
    :return: string of the most prevalent emotion, dictionary of weighted emotion sums of all faces in an image
    :rtype: str, dict
    """

    #preds is a list of dictionaries; refer to https://github.com/serengil/deepface/blob/master/deepface/DeepFace.py for detailed documentation of analyze()
    preds = DeepFace.stream(db_path = "C:\Users\jacob\Desktop\Test",  detector_backend="mtcnn")
    emotions = {}
    weights = {'sad': 1, 'angry': 1, 'surprise': 1, 'fear': 1, 'happy': 1, 'disgust': 1, 'neutral': 1}

    for pred in preds:
        #print("Face Analysis: " + str(pred))

        #reliability of the model, don't process bad data. need to check if 0.8 is too high or too low
        if pred["face_confidence"] < 0.8:
            pass
        else:
            emotions = {k: emotions.get(k, 0) + pred['emotion'].get(k, 0) for k in set(emotions) | set(pred['emotion'])}

    #weight the sums
    emotions = {k: emotions.get(k, 0) * weights.get(k, 0) for k in set(emotions) & set(weights)}

    #Determine the most prevalent emotion collected
    return max(emotions, key=emotions.get), emotions


In [15]:
import cv2
from deepface import DeepFace
from mtcnn import MTCNN
import numpy as np


In [81]:
def blur_measure(image):
    """
    Calculates the variance of Laplacian to measure image blurriness

    :param numpy.ndarray image: Input image
    :return: Variance of Laplacian
    :rtype: float
    """
    return cv2.Laplacian(image, cv2.CV_64F).var()

def emotion_faces(faces):
    """
    Determines the emotions of a collection of faces in a frame

    :param list faces: a numpy array in BGR format, base64 encoded image, or path to the image
    :return: string of the most prevalent emotion, dictionary of weighted emotion sums of all faces in an image
    :rtype: str, dict
    """

    #preds is a list of dictionaries; refer to https://github.com/serengil/deepface/blob/master/deepface/DeepFace.py for detailed documentation of analyze()
    preds = DeepFace.analyze(faces, actions=['emotion'], detector_backend="mtcnn", enforce_detection=False)
    emotions = {}
    weights = {'sad': 1, 'angry': 1, 'surprise': 1, 'fear': 1, 'happy': 1, 'disgust': 1, 'neutral': 1}

    for pred in preds:
        print("Face Analysis: " + str(pred))

        # reliability of the model, don't process bad data. need to check if 0.8 is too high or too low
        if pred["face_confidence"] < 0.8:
            pass
        else:
            emotions = {k: emotions.get(k, 0) + pred['emotion'].get(k, 0) for k in set(emotions) | set(pred['emotion'])}

    # weight the sums
    emotions = {k: emotions.get(k, 0) * weights.get(k, 0) for k in set(emotions) & set(weights)}

    # Check if emotions dictionary is empty
    if not emotions:
        return "Unknown", {}

    # Determine the most prevalent emotion collected
    return max(emotions, key=emotions.get), emotions

# Load the video
#Link to site with sample videos: https://www.pexels.com/search/videos/group%20of%20people/
video_path = "video_3.mp4"
video_capture = cv2.VideoCapture(video_path)

# Set frame rate
# Frame rate (frames per second)
fps = 2
frame_count = 0

# Create MTCNN detector
detector = MTCNN()

# Iterate through frames
timestart = time.time()
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Capture frame every 'fps' seconds
    frame_count += 1
    if frame_count % int(video_capture.get(cv2.CAP_PROP_FPS) * fps) == 0:

        print(f"Processing frame {frame_count}")

        # Detect faces
        faces = detector.detect_faces(frame)
        cropped_faces = []

        # Extract faces and append to cropped_faces list
        for face in faces:
            x, y, w, h = face['box']
            cropped_face = frame[y:y + h, x:x + w]

            # Check blurriness of the face
            #Can include this later as needed**
            # blur_num = blur_measure(cropped_face)
            # if blur_num < 100:  # Adjust threshold as needed, from testing threshold of 50 or less seems reasonable
            #     print(f"Face too blurry, skipping. measure: {blur_num}")
            #     continue  # Skip this face if too blurry

            cropped_faces.append(cropped_face)

        # Check if any faces were detected
        if len(cropped_faces) == 0:
            print("No faces detected in this frame.")
            continue  # Skip this frame if no faces are detected

        # Output the list of cropped faces
        for i, face in enumerate(cropped_faces):
            emotion, emotions = emotion_faces(face)
            if emotion:
                cv2.imwrite(f"face_{frame_count}_{i+1}_{emotion}.jpg", face)
                print(f"Emotion of face {i+1} in frame {frame_count}: {emotion}, Emotion Details: {emotions}")

video_capture.release()
cv2.destroyAllWindows()
endtime = time.time()
totaltime = endtime-timestart
print("The total processing time was " + str(totaltime))



Processing frame 50
1/1 [==============================] - 0s 35ms/step
Face Analysis: {'emotion': {'angry': 3.5029876644556906e-07, 'disgust': 4.931812571655658e-15, 'fear': 1.2366279571362582e-08, 'happy': 99.99930262561456, 'sad': 0.00020173478171354022, 'surprise': 6.234161647682429e-06, 'neutral': 0.0004884964546491607}, 'dominant_emotion': 'happy', 'region': {'x': 0, 'y': 0, 'w': 231, 'h': 307, 'left_eye': (79, 106), 'right_eye': (195, 118)}, 'face_confidence': 1.0}
Emotion of face 1 in frame 50: happy, Emotion Details: {'angry': 3.5029876644556906e-07, 'fear': 1.2366279571362582e-08, 'surprise': 6.234161647682429e-06, 'happy': 99.99930262561456, 'neutral': 0.0004884964546491607, 'sad': 0.00020173478171354022, 'disgust': 4.931812571655658e-15}
1/1 [==============================] - 0s 28ms/step
Face Analysis: {'emotion': {'angry': 1.2482612565467213e-06, 'disgust': 3.884079958637906e-12, 'fear': 9.253224860164606e-08, 'happy': 99.96469021264431, 'sad': 0.00013185726422052963, 'su

In [79]:
def blur_measure(image):
    """
    Calculates the variance of Laplacian to measure image blurriness

    :param numpy.ndarray image: Input image
    :return: Variance of Laplacian
    :rtype: float
    """
    return cv2.Laplacian(image, cv2.CV_64F).var()

def emotion_faces(faces):
    """
    Determines the emotions of a collection of faces in a frame

    :param list faces: a numpy array in BGR format, base64 encoded image, or path to the image
    :return: string of the most prevalent emotion, dictionary of weighted emotion sums of all faces in an image
    :rtype: str, dict
    """

    #preds is a list of dictionaries; refer to https://github.com/serengil/deepface/blob/master/deepface/DeepFace.py for detailed documentation of analyze()
    preds = DeepFace.analyze(faces, actions=['emotion'], detector_backend="mtcnn", enforce_detection=False)
    emotions = {}
    weights = {'sad': 1, 'angry': 1, 'surprise': 1, 'fear': 1, 'happy': 1, 'disgust': 1, 'neutral': 1}

    for pred in preds:
        print("Face Analysis: " + str(pred))

        # reliability of the model, don't process bad data. need to check if 0.8 is too high or too low
        if pred["face_confidence"] < 0.8:
            pass
        else:
            emotions = {k: emotions.get(k, 0) + pred['emotion'].get(k, 0) for k in set(emotions) | set(pred['emotion'])}

    # weight the sums
    emotions = {k: emotions.get(k, 0) * weights.get(k, 0) for k in set(emotions) & set(weights)}

    # Check if emotions dictionary is empty
    if not emotions:
        return "Unknown", {}

    # Determine the most prevalent emotion collected
    return max(emotions, key=emotions.get), emotions

# Load the video
#Link to site with sample videos: https://www.pexels.com/search/videos/group%20of%20people/
video_path = "video_3.mp4"
video_capture = cv2.VideoCapture(video_path)

# Set frame rate
# Frame rate (frames per second)
fps = 2
frame_count = 0

# Create MTCNN detector
detector = MTCNN()

# Iterate through frames
timestart = time.time()
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Capture frame every 'fps' seconds
    frame_count += 1
    if frame_count % int(video_capture.get(cv2.CAP_PROP_FPS) * fps) == 0:

        print(emotion_faces(frame))
video_capture.release()
cv2.destroyAllWindows()
endtime = time.time()
totaltime = endtime-timestart
print("The total processing time was " + str(totaltime))



2/2 [==============================] - 0s 11ms/step
Face Analysis: {'emotion': {'angry': 8.699236331288773e-08, 'disgust': 9.69217517658369e-12, 'fear': 5.229933153216848e-08, 'happy': 99.78564381599426, 'sad': 8.835223184178176e-05, 'surprise': 1.1515107090076526e-06, 'neutral': 0.21426640450954437}, 'dominant_emotion': 'happy', 'region': {'x': 524, 'y': 311, 'w': 237, 'h': 333, 'left_eye': (607, 436), 'right_eye': (724, 450)}, 'face_confidence': 1.0}
Face Analysis: {'emotion': {'angry': 1.7751172753932565e-08, 'disgust': 1.236636833302302e-19, 'fear': 1.160357210707244e-12, 'happy': 100.0, 'sad': 1.0675012873662126e-06, 'surprise': 7.027192716613229e-09, 'neutral': 2.3983302455121702e-06}, 'dominant_emotion': 'happy', 'region': {'x': 946, 'y': 177, 'w': 225, 'h': 298, 'left_eye': (1018, 300), 'right_eye': (1118, 299)}, 'face_confidence': 1.0}
Face Analysis: {'emotion': {'angry': 1.5129628959397342e-15, 'disgust': 2.6183079921519256e-22, 'fear': 4.2373082396893234e-10, 'happy': 99.712